In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import numpy as np
import pandas as pd


import CBOEFileParser
import VerticalSpreads
import CBOE_Downloader
import POP

from pymongo import MongoClient

In [2]:
# underlying_symbol = "BIDU"
# risk_free_rate = 3.0
# risk_limit = 500

In [3]:
# pop_calculator = POP.POP().get_ohlc_data(underlying_symbol)

In [7]:
underlying_symbol = "GOOG"
risk_limit = 200
risk_free_rate = 3.0

mongo_address = "mongodb://localhost:27017"

MongoClient(mongo_address).Spreads.vertical.delete_many({})

pop_calculator = POP.POP().get_ohlc_data(underlying_symbol)
# cboe_file = CBOE_Downloader.CBOE_Downloader(underlying_symbol).download().save_download().get_filename()
# cboe_file = "/Users/glenn/Documents/CBOE_Data/BIDU_201807261013.dat"
cboe_file = "/Users/glenn/Documents/CBOE_Data/GOOG_201807301217.dat"

option_chain_obj = CBOEFileParser.CBOEFileParser(
                    remove_options_after_days=60,
                    remove_options_before_days=30,
                    set_exchange=None) \
                    .parse_cboe_file(cboe_file=cboe_file) \
                    .add_greeks(risk_free_rate=0.298)

for expiration, calls, puts in option_chain_obj.items():
    print("Expiration: {}".format(expiration))
    print("Calls: {:,}".format(len(calls)))
    print("Puts: {:,}".format(len(puts)))
    
vertical_spreads = VerticalSpreads.VerticalSpreads() \
                                    .build_spreads(option_chain_obj) \
                                    .risk_limit(risk_limit)\
                                    .itm_spreads(option_chain_obj.underlying_spot) \
                                    .sky_view_spreads() \
                                    .add_pop(pop_calculator, option_chain_obj.underlying_spot) \
                                    .add_expected_return(pop_calculator, option_chain_obj.underlying_spot) \
                                    .write_to_mongo(mongo_address=mongo_address)
                
print("Current Spot: {}".format(option_chain_obj.underlying_spot))
for expiration, bull_calls, bull_puts, bear_calls, bear_puts in vertical_spreads.items():
    print("Expiration: {}".format(expiration))
    print("Bull Puts: {:,}".format(len(bull_puts)))
    print("Bear Calls: {:,}".format(len(bear_calls)))

Expiration: 2018-08-31 23:59:00
Calls: 116
Puts: 114
Expiration: 2018-09-07 23:59:00
Calls: 84
Puts: 81
Expiration: 2018-09-21 23:59:00
Calls: 161
Puts: 124
Current Spot: 1224.99
Expiration: 2018-08-31 23:59:00
Bull Puts: 46
Bear Calls: 63
Expiration: 2018-09-07 23:59:00
Bull Puts: 14
Bear Calls: 60
Expiration: 2018-09-21 23:59:00
Bull Puts: 26
Bear Calls: 27
